In [1]:
import os
from dotenv import load_dotenv
# Loads environment variables from a .env file into your system's environment.
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq

model = ChatGroq(api_key=groq_api_key, model="Gemma2-9b-it")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000025FF32B5A00>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000025FF32B6BA0>, model_name='Gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

## PromptTemplate
Prompt Template helps to turn raw user information into a format that the LLM (Large Language Model) can understand and reason over effectively.
It allows developers to structure inputs using placeholders and inject dynamic values (like user queries, retrieved documents, or memory) into a predefined message layout — ensuring consistent, contextual, and optimized prompts for better responses.

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage

# The "system" message sets the role and behavior of the assistant, providing context for how it should respond.
# MessagesPlaceholder is a dynamic placeholder that allows injecting a list of messages (conversation history) at runtime.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are helpful assistant. Answer all the question to the best of your ability in this {language}."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

parser = StrOutputParser()
chain = prompt|model|parser

In [4]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# This is an in-memory store that maps each unique session ID (like a user's chat session) to its own ChatMessageHistory.
# Think of it like a cache that holds ongoing conversations by session_id.
store = {}

# This function checks if a message history exists for the given session, If not, it initializes a new ChatMessageHistory() object and stores it in store using the session_id as the key
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [5]:
# Session config, It creates a config dictionary that is passed to .invoke() in RunnableWithMessageHistory. This tells LangChain which conversation session to use
config = {"configurable":{"session_id":"chat1"}}

## Summary of `trim_messages` in LangChain

`trim_messages` is a utility that helps **manage the size of the message history** being sent to the LLM. It ensures that the **total token count** of the messages remains **within a specified limit**, like `max_tokens=200`. You can configure:
- `strategy="last"` – Keep the most recent messages.
- `include_system=True` – Retain system prompts.
- `allow_partial=False` – Only complete messages are kept (no cutting in the middle).
- `start_on="human"` – Decide from which type of message trimming should start.


In [6]:
from langchain_core.messages import trim_messages
# Trimmer for managing token limits
trimmer = trim_messages(
    max_tokens=500,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

#### What is `RunnablePassthrough` in LangChain?

`RunnablePassthrough` is a **utility runnable** in LangChain that:

**Simply returns the input as-is** without modifying it.

Think of it as a "pass-through" pipe — it doesn't change the data but allows you to **chain operations on the input** like `.assign(...)`.

---

### 1. `RunnablePassthrough.assign(...)`

This means:
- You’re **starting with the full input dictionary**.
- You're adding/modifying a field called `messages`.
- You compute `messages` by:
  ```python
  itemgetter("messages") | trimmer
  ```
  So:
  - `itemgetter("messages")`: extracts the `"messages"` key from the input.
  - `trimmer`: trims that message list based on token constraints.
  - `|` is LangChain's way of chaining operations.

So you're **replacing/setting the `messages` field in the input with its trimmed version**.

---

###  Without `RunnablePassthrough`?

You’d have to handle this logic outside the chain or write custom wrappers — so it gives **clean composition and readability**.

In [7]:
# Create the actual chain
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
    | parser
)


In [8]:
# Wrap chain with message history tracking
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [9]:
response = with_message_history.invoke(
    {'messages': [HumanMessage(content="My Name is Ravi")],"language":"french"},
    config=config
)
print(response)

Enchanté, Ravi !  

Je suis ici pour vous aider. N'hésitez pas à me poser toutes vos questions en français. Je ferai de mon mieux pour y répondre. 😊 



In [10]:
response = with_message_history.invoke(
    {'messages': [HumanMessage(content="I am a software developer and aspire to become AI Engineer")],"language":"english"},
    config=config
)
print(response)

That's a fantastic goal, Ravi!  Software development is a great foundation for becoming an AI Engineer. 

What aspects of AI engineering are you most interested in?  

Knowing your specific interests will help me give you more tailored advice.  For example, are you drawn to:

* **Machine Learning?** (building algorithms that learn from data)
* **Deep Learning?** (a subset of machine learning using artificial neural networks)
* **Natural Language Processing?** (making computers understand and generate human language)
* **Computer Vision?** ( enabling computers to "see" and interpret images)
* **Robotics?** (designing and building intelligent robots)


Tell me more about your aspirations, and let's explore the path to becoming an AI Engineer together!



In [11]:
response = with_message_history.invoke(
    {'messages': [HumanMessage(content="Whats my name")],"language":"hindi"},
    config=config
)
print(response)

Your name is Ravi!  😊  I remember that you told me at the beginning.  






